In [1]:
from src.modules.vin_dataclass import Vin
import src.modules.data_extractor as de
from bs4 import BeautifulSoup as BS
from pathlib import Path
from serde.json import to_json
import polars as pl

from dataclasses import dataclass
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

import numpy as np
import re

In [26]:
def scrapeur(lien : str): #"https://www.vinatis.com/achat-vin-rouge"
    ## LANCE CHROME ##
    service = Service(executable_path='./chromedriver.exe')
    driver = webdriver.Chrome(service=service)
    driver.get(lien)

    ## FERME LES COOKIES ##
    cookie_accept_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Accepter et continuer')]")
    cookie_accept_button.click()

    ## FERME POPUP ## pop-up "Inscription" apparait de manière aléatoire
    while True:
        try:
            time.sleep(10)
            driver.find_element(By.CLASS_NAME, 'close_modal_vinatis').click()
        except :
            continue  
        break 

    ## RECUPERE NB PAGE à SCRAPER##
    page_source = driver.page_source
    soupe = BS(page_source)
    result_nb_pages = soupe.find_all(name="span", attrs={"class": "margin-right-10"})
    NOMBRE_PAGE = int(re.findall(r'\d+', result_nb_pages[0].text)[0])

    ## BOUCLE POUR PARCOURIR LES PAGES ##
    i = 1
    vins = list()
    while i < NOMBRE_PAGE:
        ## SCRAPING ##
        page_source = driver.page_source
        soupe = BS(page_source)
        
        for (item_1, item_2, item_3, item_4) in zip(
            de.extract_result(soupe)["result_title"], 
            de.extract_result(soupe)["result_price"], 
            de.extract_result(soupe)["result_avis"],
            de.extract_result(soupe)["result_type"]
        ):
            vins.append(de.extract_vin(item_1, item_2, item_3, item_4))
        
        ## CHANGEMENT DE PAGE ##
        time.sleep(2)
        bouton = driver.find_element(By.CLASS_NAME, 'icon-chevron-right')
        time.sleep(2)
        driver.execute_script("arguments[0].scrollIntoView();", bouton)
        time.sleep(2)
        driver.execute_script("window.scrollBy(0, -500);")
        time.sleep(2)
        bouton.click()
        time.sleep(2)
        i = i + 1

    ## FERME CHROME ##
    driver.quit()
    
    return vins


In [29]:
vins = scrapeur("https://www.vinatis.com/champagne-brut")

In [30]:
vins_json = to_json(vins)

In [31]:
file_path = "data/vins.json"

with open(file_path, "w",  encoding='utf-8') as json_file:
    json_file.write(vins_json)

In [32]:
pl.read_json(file_path)

name,capacity,price,promo,prix_promo,note,nb_avis,type,year
str,f64,f64,str,f64,f64,i64,str,i64
"""CHAMPAGNE JEAN…",0.75,29.9,"""-27%""",21.7,16.7,115,"""x""",null
"""CHAMPAGNE CL D…",0.75,620.5,"""-14%""",null,16.8,34,"""x""",null
"""CHAMPAGNE HATO…",0.75,221.5,"""-12%""",null,16.4,159,"""x""",null
"""CHAMPAGNE ANGE…",0.75,69.9,"""-17%""",57.9,16.8,12,"""x""",null
"""CHAMPAGNE LECL…",0.75,49.9,"""-20%""",39.9,17.1,38,"""x""",null
"""CHAMPAGNE MUMM…",0.75,249.9,"""-11%""",null,18.3,11,"""x""",null
"""CHAMPAGNE MUMM…",0.75,24.9,null,null,18.3,3,"""x""",null
"""CHAMPAGNE MOET…",0.75,58.9,null,null,19.1,9,"""x""",null
"""CHAMPAGNE MICH…",0.75,240.9,null,null,16.8,87,"""x""",null
